In [0]:


from itertools import product
import datetime
import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002, 1000]
IOS_DEVICE_ID = [2001, 2002, 2000]
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
DATE = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

time = time.strftime('%Y-%m-%d_%H-%M-%S',time.localtime(time.time()))
log_filename = '/tmp/db_{time}.log'.format(time=time)
print log_filename

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_key_product_list(granularity=False, device_id=False, store_id=False, network_id=False):
    granularity_list = GRANULARIY if granularity else [None]
    device_id_list = GP_DEVICE_ID + IOS_DEVICE_ID if device_id else [None]
    store_id_list = GP_COUNTRY_CODE + IOS_COUNTRY_CODE if store_id else [None]
    network_id_list = NETWORK if network_id else [None]
    product_list = []
    for g, d, s, n in product(granularity_list, device_id_list, store_id_list, network_id_list):
        if (d in GP_DEVICE_ID and s in IOS_COUNTRY_CODE) or (d in IOS_DEVICE_ID and s in GP_COUNTRY_CODE):
            continue
        product_list.append((g,d,s,n))
    return product_list

def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)


def check_mkt_data(table_name, table_keys, is_facet, **kw):
    sql_where = "creative_id <= 90100006099406" if "creative_id" in table_keys else "date < '2019-05-01'"
    for granularity,device_id,store_id, network_id in get_key_product_list(**kw):
        # SQL RESULT
        result = 0
        error_result = 0
        sql_table = "{granularity}{table_name}{device_id}{store_id}".format(
            table_name=table_name + "_",
            device_id=str(device_id) + "_" if device_id else "",
            store_id=str(store_id) if store_id else "",
            granularity=str(granularity) + "_" if granularity else "")
        # print sql_table
        for host, port in dblist:
            dbinfo['HOST'] = host
            dbinfo['PORT'] = port
            try:
                sql =  "SELECT COUNT(DISTINCT ({table_keys}) ) FROM mkt.{sql_table} WHERE {sql_where}".format(
                    table_keys=table_keys, sql_table=sql_table, sql_where=sql_where)
                r, _ = select(sql)
                result += r[0][0]
            except Exception, e:
                print e.message
                error_result=e.message.replace('\n', '\\n')
                if "does not exist" in error_result:
                    error_result = 0
        # S3 RESULT
        s3_result = 0
        error_s3_result = 0
        s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/{fact}/{granularity}{device_id}{store_id}".format(
            table_name=table_name, fact='fact' if is_facet else 'dimension',
            granularity = "granularity={}/".format(granularity) if granularity else "",
            device_id = "device_id={}/".format(device_id) if device_id else "",
            store_id = "store_id={}/".format(store_id)) if store_id else ""
        # print s3_path
        try:
            s3_result = spark.read.parquet(s3_path).count()
        except Exception, e:
            error_s3_result = 0

        result_str = "{},{},{}\n".format(sql_table, result or error_result,s3_result or error_s3_result)
        write_to_file(log_filename, result_str)
        # print result_str
        # return

# check_mkt_data("meta_creative", "device_id, store_id, network_id, creative_id, ad_app_id, first_seen, last_seen", is_facet=False, device_id=True, store_id=True )
# TBD check_mkt_data("meta_creative_pub", "device_id , store_id , date , network_id , creative_id , ad_app_id , pub_app_id", is_facet=False, device_id=True, store_id=True )
# TBD check_mkt_data("meta_creative_campaign", "device_id , store_id , date , network_id , creative_id , ad_app_id , campaign_id", is_facet=False, device_id=True, store_id=True )

# TBD check_mkt_data("creative_estimate", "device_id , store_id , date , network_id , ad_app_id , creative_id , kpi, estimate", is_facet=True, granularity=True, device_id=True, store_id=True )
# TBD check_mkt_data("creative_detail_estimate", "device_id , store_id , date , network_id , ad_app_id , creative_id , kpi, id_list", is_facet=True, granularity=True, device_id=True, store_id=True )
# TBD check_mkt_data("creative_ad_estimate", "device_id , store_id , date , network_id , ad_app_id , creative_id , estimate", is_facet=True, granularity=True, device_id=True, store_id=True )

check_mkt_data("ad_app_estimate", "device_id , store_id , network_id , date , ad_app_id , estimate , kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("ad_app_detail_estimate", "device_id , store_id , network_id , date , ad_app_id , id_list , kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("ad_app_pub_app_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_app_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# TBD check_mkt_data("pub_app_ad_app_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_app_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# NOT READY check_mkt_data("pub_app_ad_app_detail_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_app_id , id_list, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("ad_app_pub_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("pub_app_estimate", "device_id , store_id , date , network_id , pub_app_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("pub_app_detail_estimate", "device_id , store_id , date , network_id , pub_app_id , id_list ,kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )

#  ???? no date or creative id check_mkt_data("pub_app_ad_app_date_range_estimate", "device_id , store_id , network_id , pub_app_id , ad_app_id , first_seen , last_seen", is_facet=False, device_id=True, store_id=True )

check_mkt_data("creative_category", "device_id , store_id , category_id , creative_id , ad_app_id , date , rank , network_id",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("ad_app_category", "device_id , store_id , category_id , ad_app_id , date , rank , network_id",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("pub_app_category", "device_id , store_id , category_id , pub_app_id , date , rank , network_id",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("adpo_estimate", "device_id , store_id , network_id , category_id , date , kpi , estimate",  is_facet=True, granularity=True,device_id=True, store_id=True )
check_mkt_data("adpo_detail_estimate", "device_id , store_id , network_id , category_id , date , kpi , estimate",  is_facet=True, granularity=True,device_id=True, store_id=True )




In [0]:


from itertools import product
import datetime
import threading

import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95, 1000]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002, ] # todo 1000
IOS_DEVICE_ID = [2001, 2002, ] # todo 2000
GRANULARIY = ["weekly", "monthly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
MONTH = ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_key_product_list(granularity=False, device_id=False, store_id=False, network_id=False, month=False):
    granularity_list = GRANULARIY if granularity else [None]
    device_id_list = GP_DEVICE_ID + IOS_DEVICE_ID if device_id else [None]
    store_id_list = GP_COUNTRY_CODE + IOS_COUNTRY_CODE if store_id else [None]
    network_id_list = NETWORK if network_id else [None]
    month_list = MONTH if month else [None]
    product_list = []
    for g, d, s, n, m in product(granularity_list, device_id_list, store_id_list, network_id_list, month_list):
        if (d in GP_DEVICE_ID and s in IOS_COUNTRY_CODE) or (d in IOS_DEVICE_ID and s in GP_COUNTRY_CODE):
            continue
        product_list.append((g,d,s,n, m))
    return product_list

def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

def check_mkt_data(table_name, table_keys, is_facet):
    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime(time.time()))
    log_filename = '/tmp/db_{}_{}.log'.format(timestamp, threading.currentThread().name)
    print log_filename
    while (len(key_product_list) > 0):
        time_start = time.time()
        granularity, device_id, store_id, network_id, month = key_product_list.pop()
        sql_where = "creative_id <= 90100006099406" if "creative_id" in table_keys else "date < '2019-05-01'"
        # SQL RESULT
        result = 0
        error_result = 0
        sql_table = "{granularity}{table_name}{device_id}{store_id}{network_id}{month}".format(
            table_name=table_name + "_",
            device_id=str(device_id) + "_" if device_id else "",
            store_id=str(store_id) + "_" if store_id else "",
            granularity=str(granularity) + "_" if granularity else "",
            month = str(month) + "_" if month else "",
            network_id = str(network_id) + "_" if  network_id else "").rstrip('_')
        # print sql_table
        for host, port in dblist:
            dbinfo['HOST'] = host
            dbinfo['PORT'] = port
            try:
                sql =  "SELECT COUNT(DISTINCT ({table_keys}) ) FROM mkt.{sql_table} WHERE {sql_where}".format(
                    table_keys=table_keys, sql_table=sql_table, sql_where=sql_where)
                r, _ = select(sql)
                result += r[0][0]
            except Exception, e:
                # print e.message
                error_result=e.message.replace('\n', '\\n')
                if "does not exist" in error_result:
                    error_result = 0
        # S3 RESULT
        s3_result = 0
        error_s3_result = 0
        s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/{fact}/{month}{granularity}{device_id}{store_id}{network_id}".format(
            table_name=table_name, fact='fact' if is_facet else 'dimension',
            month = "month={}/".format(str('-'.join([month[:4], month[-2:]]))) if month else "",
            granularity = "granularity={}/".format(granularity) if granularity else "",
            device_id = "device_id={}/".format(device_id) if device_id else "",
            store_id = "store_id={}/".format(store_id) if store_id else "",
            network_id = "network_id={}/".format(network_id) if network_id else "")
        try:
            s3_result = spark.read.parquet(s3_path).count()
            # print s3_path
        except Exception, e:
            error_s3_result = 0

        time_end = time.time()
        result_str = "{},{},{},{}\n".format(sql_table, result or error_result,s3_result or error_s3_result, int(time_end-time_start))
        write_to_file(log_filename, result_str)


print len(get_key_product_list(device_id=True, store_id=True, network_id=True, month=True))

thread_count = 10
# key_product_list = [(None, "2002","143441","1","201712")] # DEBUG granularity, device_id, store_id, network_id, month
key_product_list = get_key_product_list(device_id=True, network_id=True, store_id=True, month=True)

thread_group = []
for index in range(0,thread_count):
    t = threading.Thread(
        name="thread-{}".format(index),
        target=check_mkt_data,
        args=(
        "meta_creative_pub", "device_id , store_id , date , network_id , creative_id , ad_app_id , pub_app_id", True))
    thread_group.append(t)
    t.start()

for t in thread_group:
    t.join()


# # check_mkt_data("meta_creative", "device_id, store_id, network_id, creative_id, ad_app_id, first_seen, last_seen", is_facet=False, device_id=True, store_id=True )
# # TBD check_mkt_data("meta_creative_pub", "device_id , store_id , date , network_id , creative_id , ad_app_id , pub_app_id", is_facet=False, device_id=True, store_id=True )
# # TBD check_mkt_data("meta_creative_campaign", "device_id , store_id , date , network_id , creative_id , ad_app_id , campaign_id", is_facet=False, device_id=True, store_id=True )
#
# # TBD check_mkt_data("creative_estimate", "device_id , store_id , date , network_id , ad_app_id , creative_id , kpi, estimate", is_facet=True, granularity=True, device_id=True, store_id=True )
# # TBD check_mkt_data("creative_detail_estimate", "device_id , store_id , date , network_id , ad_app_id , creative_id , kpi, id_list", is_facet=True, granularity=True, device_id=True, store_id=True )
# # TBD check_mkt_data("creative_ad_estimate", "device_id , store_id , date , network_id , ad_app_id , creative_id , estimate", is_facet=True, granularity=True, device_id=True, store_id=True )
#
# check_mkt_data("ad_app_estimate", "device_id , store_id , network_id , date , ad_app_id , estimate , kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("ad_app_detail_estimate", "device_id , store_id , network_id , date , ad_app_id , id_list , kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("ad_app_pub_app_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_app_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# # TBD check_mkt_data("pub_app_ad_app_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_app_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# # NOT READY check_mkt_data("pub_app_ad_app_detail_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_app_id , id_list, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("ad_app_pub_estimate", "device_id , store_id , date , network_id , ad_app_id , pub_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("pub_app_estimate", "device_id , store_id , date , network_id , pub_app_id , estimate, kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("pub_app_detail_estimate", "device_id , store_id , date , network_id , pub_app_id , id_list ,kpi",  is_facet=True, granularity=True,device_id=True, store_id=True )
#
# #  ???? no date or creative id check_mkt_data("pub_app_ad_app_date_range_estimate", "device_id , store_id , network_id , pub_app_id , ad_app_id , first_seen , last_seen", is_facet=False, device_id=True, store_id=True )
#
# check_mkt_data("creative_category", "device_id , store_id , category_id , creative_id , ad_app_id , date , rank , network_id",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("ad_app_category", "device_id , store_id , category_id , ad_app_id , date , rank , network_id",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("pub_app_category", "device_id , store_id , category_id , pub_app_id , date , rank , network_id",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("adpo_estimate", "device_id , store_id , network_id , category_id , date , kpi , estimate",  is_facet=True, granularity=True,device_id=True, store_id=True )
# check_mkt_data("adpo_detail_estimate", "device_id , store_id , network_id , category_id , date , kpi , estimate",  is_facet=True, granularity=True,device_id=True, store_id=True )
#
#


In [0]:
%%sh

# 459680
# export FILE="/tmp/db_2019-08-20_05-53-35*"
# cat $FILE | grep -v '0,0,'
#echo $(((wc -l $FILE)/459680))
# wc -l $FILE
# cat $FILE | grep '2002_143441_1_201712'

# -rw-r--r--  1 hadoop   hadoop   125910 Aug 19 23:16 db_2019-08-19_17-54-37.log
# -rw-r--r--  1 hadoop   hadoop   126089 Aug 19 23:20 db_2019-08-19_17-54-37.log
# -rw-r--r--  1 hadoop   hadoop   153486 Aug 20 01:52 db_2019-08-19_17-54-37.log
# ls $FILE

# cat  /tmp/db_2019-08-20_05-53-35_thread-0.log | grep -v '0,0,'
#  cat /tmp/db_2019-08-20_05-53-35_thread-1.log | grep -v '0,0,'
# cat /tmp/db_2019-08-20_05-53-35_thread-2.log | grep -v '0,0,'
# cat /tmp/db_2019-08-20_05-53-35_thread-3.log | grep -v '0,0,'
# cat /tmp/db_2019-08-20_05-53-35_thread-4.log | grep -v '0,0,'
# cat /tmp/db_2019-08-20_05-53-35_thread-5.log | grep -v '0,0,'
cat /tmp/db_2019-08-20_05-53-35_thread-6.log | grep -v '0,0,'
cat /tmp/db_2019-08-20_05-53-35_thread-7.log | grep -v '0,0,'
cat /tmp/db_2019-08-20_05-53-35_thread-8.log | grep -v '0,0,'
cat /tmp/db_2019-08-20_05-53-35_thread-9.log | grep -v '0,0,'



In [0]:
%%sh
ls -al /tmp/


for VARIABLE in 0 1 2 3 4 5 6 7 8 9
do
	export LOGFILE="/tmp/db_2019-08-20_05-41-11_thread-"$VARIABLE".log"
    rm $LOGFILE  & mkdir -p $LOGFILE
done


In [0]:


from itertools import product
import threading

import psycopg2
from contextlib import closing
import time

GP_COUNTRY_CODE = [1000, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95]
IOS_COUNTRY_CODE = [0, 143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]
GP_DEVICE_ID = [1001, 1002, 1000]
IOS_DEVICE_ID = [2001, 2002, 2000]
GRANULARIY = ["monthly", "weekly"]
NETWORK = [1, 203, 324, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 114, 116, 142, 311, 121, 316, 325]
MONTH = ['201904', '201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903']

dbinfo = {'NAME': 'mkt_est', 'USER': 'app_bdp_usage_qa', 'PASSWORD': '2mHdFW6%#REu', 'HOST': '10.2.6.27', 'PORT': 5433}
dblist = [('10.2.6.27', 5433), ('10.2.6.245', 5434), ('10.2.6.42', 5435), ('10.2.6.148', 5436), ('10.2.6.49', 5437), ('10.2.6.52', 5438), ('10.2.6.187', 5439), ('10.2.6.254', 5440)]

def select(sql):
    conn = psycopg2.connect(dbname=dbinfo['NAME'], user=dbinfo['USER'], password=dbinfo['PASSWORD'],host=dbinfo['HOST'], port=dbinfo['PORT'])
    with closing(conn.cursor()) as cur:
        cur.execute(sql)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def get_key_product_list(granularity=False, device_id=False, store_id=False, network_id=False, month=False):
    granularity_list = GRANULARIY if granularity else [None]
    device_id_list = GP_DEVICE_ID + IOS_DEVICE_ID if device_id else [None]
    store_id_list = GP_COUNTRY_CODE + IOS_COUNTRY_CODE if store_id else [None]
    network_id_list = NETWORK if network_id else [None]
    month_list = MONTH if month else [None]
    product_list = []
    for g, d, s, n, m in product(granularity_list, device_id_list, store_id_list, network_id_list, month_list):
        if (d in GP_DEVICE_ID and s in IOS_COUNTRY_CODE) or (d in IOS_DEVICE_ID and s in GP_COUNTRY_CODE):
            continue
        product_list.append((g,d,s,n, m))
    return product_list

def write_to_file(filename, str):
    with open(filename,'a') as file_object:
        file_object.write(str)

def get_where(table_keys):
    where_condition = []
    if "creative_id" in table_keys:
        where_condition.append("creative_id <= 90100006099406")
    if "kpi" in table_keys:
        where_condition.append( "kpi IS NOT NULL")
    if "update_date" in table_keys:
        where_condition.append("update_date < '2019-05-01'")
    elif "date" in table_keys:
        where_condition.append("date < '2019-05-01'")
    if len(where_condition)==0:
        return "TRUE"
    else:
        return " AND ".join(where_condition)



def _check_mkt_data_thread(table_name, table_keys, is_facet):
    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime(time.time()))
    log_filename = '/tmp/db_test/{}_{}_{}.log'.format(table_name, timestamp, threading.currentThread().name)
    print log_filename
    while (len(key_product_list) > 0):
        time_start = time.time()
        keys_tuple = key_product_list.pop()
        granularity, device_id, store_id, network_id, month = keys_tuple
        sql_where=get_where(table_keys)
        # SQL RESULT
        result = 0
        error_result = 0
        sql_table = "{granularity}{table_name}{device_id}{store_id}{network_id}{month}".format(
            table_name=table_name + "_",
            device_id=str(device_id) + "_" if device_id else "",
            store_id=str(store_id) + "_" if store_id else "",
            granularity=str(granularity) + "_" if granularity else "",
            month = str(month) + "_" if month else "",
            network_id = str(network_id) + "_" if  network_id else "").rstrip('_')
        # print sql_table
        for host, port in dblist:
            dbinfo['HOST'] = host
            dbinfo['PORT'] = port
            try:
                sql =  "SELECT COUNT(DISTINCT ({table_keys}) ) FROM mkt.{sql_table} WHERE {sql_where}".format(
                    table_keys=table_keys, sql_table=sql_table, sql_where=sql_where)
                r, _ = select(sql)
                result += r[0][0]
            except Exception, e:
                # print e.message
                error_result=e.message.replace('\n', '\\n')
                if "does not exist" in error_result:
                    error_result = 0
                    continue
                key_product_list.append(keys_tuple)
        # S3 RESULT
        s3_result = 0
        error_s3_result = 0
        s3_path = "s3://b2c-prod-data-pipeline-unified-market/unified/app-tech.market.legacy-{table_name}.v1/{fact}/{month}{granularity}{device_id}{store_id}{network_id}".format(
            table_name=table_name, fact='fact' if is_facet else 'dimension',
            month = "month={}/".format(str('-'.join([month[:4], month[-2:]]))) if month else "",
            granularity = "granularity={}/".format(granularity) if granularity else "",
            device_id = "device_id={}/".format(device_id) if device_id else "",
            store_id = "store_id={}/".format(store_id) if store_id else "",
            network_id = "network_id={}/".format(network_id) if network_id else "")
        try:
            s3_result = spark.read.parquet(s3_path).count()
            # print s3_path
        except Exception, e:
            error_s3_result = 0

        time_end = time.time()
        result_str = "{},{},{},{},{}\n".format(table_name, sql_table, result or error_result,s3_result or error_s3_result, int(time_end-time_start))
        write_to_file(log_filename, result_str)


# print len(get_key_product_list(device_id=True, store_id=True, network_id=True, month=True))

thread_count = 10
# key_product_list = [(None, "2002","143441","1","201712")] # DEBUG granularity, device_id, store_id, network_id, month
key_product_list = []


def check_mkt_data(table_name, table_keys, is_facet, **kw):
    thread_group = []
    global key_product_list
    key_product_list = get_key_product_list(**kw)
    print len(key_product_list)
    for index in range(0,thread_count):
        t = threading.Thread(
            name="thread-{}".format(index),
            target=_check_mkt_data_thread,
            args=(table_name, table_keys, is_facet))
        thread_group.append(t)
        t.start()

    for t in thread_group:
        t.join()




#
check_mkt_data("adpo_estimate", "device_id , store_id , network_id , category_id , date , kpi , estimate",  is_facet=True, granularity=True,device_id=True, )
check_mkt_data("adpo_detail_estimate", "device_id , store_id , network_id , category_id , date , kpi , estimate",  is_facet=True, granularity=True,device_id=True)



In [0]:
%%sh
date
ls -al /tmp/db_test/*
# wl -c /tmp/db_test/*
cat /tmp/db_test/* | grep -v '0,0,'
# mkdir -p /tmp/db_test/
# rm -rf /tmp/db_test/*

# for VARIABLE in 0 1 2 3 4 5 6 7 8 9
# do
# 	export LOGFILE="/tmp/db_new_ad_app_estimate_2019-08-20_15-14-55_thread-"$VARIABLE".log"
#     rm $LOGFILE  & mkdir -p $LOGFILE
# done


In [0]:
%%sh


cat /tmp/db_test/adpo_detail_estimate_2019-08-21_06-18-18_thread-0.log


In [0]:
%%sh
